<a href="https://colab.research.google.com/github/Ian-MichaelOFF/Trabajos-Collab/blob/main/Codigo%20Proyecto%20Correo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install pandas
!pip install streamlit
!pip install pyngrok


In [34]:
import pandas as pd
from datetime import datetime
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pickle
import os

# Archivos de datos
BIRTHDAYS_FILE = 'birthdays.csv'
MESSAGES_FILE = 'messages.pkl'

# Configuración de correo
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

# Funciones auxiliares
def load_birthdays():
    if os.path.exists(BIRTHDAYS_FILE):
        return pd.read_csv(BIRTHDAYS_FILE)
    else:
        return pd.DataFrame(columns=["Name", "Birthdate", "Email"])


def save_birthdays(data):
    data.to_csv(BIRTHDAYS_FILE, index=False)


def load_messages():
    if os.path.exists(MESSAGES_FILE):
        with open(MESSAGES_FILE, 'rb') as f:
            return pickle.load(f)
    else:
        return []


def save_messages(messages):
    with open(MESSAGES_FILE, 'wb') as f:
        pickle.dump(messages, f)


def calculate_days_to_birthday(birthdate):
    today = datetime.now().date()
    birthdate = datetime.strptime(birthdate, "%Y-%m-%d").date()
    next_birthday = birthdate.replace(year=today.year)
    if next_birthday < today:
        next_birthday = next_birthday.replace(year=today.year + 1)
    return (next_birthday - today).days


def send_email(to_email, subject, body, email_user, email_password):
    try:
        msg = MIMEMultipart()
        msg["From"] = email_user
        msg["To"] = to_email
        msg["Subject"] = subject
        msg.attach(MIMEText(body, "plain"))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(email_user, email_password)
            server.send_message(msg)
        print(f"Correo enviado a {to_email}.")
        return True
    except Exception as e:
        print(f"Error al enviar correo: {e}")
        return False


# Funciones principales
def register_birthday():
    name = input("Nombre: ")
    birthdate = input("Fecha de Nacimiento (YYYY-MM-DD): ")
    email = input("Correo Electrónico: ")
    new_entry = {"Name": name, "Birthdate": birthdate, "Email": email}
    birthdays = load_birthdays()
    birthdays = pd.concat([birthdays, pd.DataFrame([new_entry])], ignore_index=True)
    save_birthdays(birthdays)
    print("Cumpleaños registrado con éxito.")


def view_upcoming_birthdays():
    birthdays = load_birthdays()
    if birthdays.empty:
        print("No hay cumpleaños registrados.")
        return
    birthdays["Days Until"] = birthdays["Birthdate"].apply(calculate_days_to_birthday)
    birthdays = birthdays.sort_values("Days Until")
    print(birthdays[["Name", "Birthdate", "Days Until"]])


def manage_messages():
    messages = load_messages()
    print("Mensajes actuales:")
    for i, msg in enumerate(messages, 1):
        print(f"{i}. {msg}")
    new_message = input("Escribe un nuevo mensaje (o presiona Enter para omitir): ")
    if new_message:
        messages.append(new_message)
        save_messages(messages)
        print("Mensaje agregado.")


def send_todays_emails(email_user, email_password):
    today = datetime.now().strftime("%Y-%m-%d")
    birthdays = load_birthdays()
    birthdays_today = birthdays[birthdays["Birthdate"].str.endswith(today[-5:])]
    messages = load_messages()
    if birthdays_today.empty:
        print("No hay cumpleaños hoy.")
        return
    for _, row in birthdays_today.iterrows():
        name, email = row["Name"], row["Email"]
        message = messages[0] if messages else f"¡Feliz cumpleaños, {name}!"
        send_email(email, "¡Feliz Cumpleaños!", message, email_user, email_password)


# Menú principal
def main():
    print("=== Gestor de Cumpleaños ===")
    email_user = input("Introduce tu correo electrónico: ")
    email_password = input("Introduce tu contraseña: ")

    while True:
        print("\nOpciones:")
        print("1. Registrar Cumpleaños")
        print("2. Ver Próximos Cumpleaños")
        print("3. Gestionar Mensajes")
        print("4. Enviar Correos de Hoy")
        print("5. Salir")

        choice = input("Elige una opción: ")

        if choice == "1":
            register_birthday()
        elif choice == "2":
            view_upcoming_birthdays()
        elif choice == "3":
            manage_messages()
        elif choice == "4":
            send_todays_emails(email_user, email_password)
        elif choice == "5":
            print("¡Adiós!")
            break
        else:
            print("Opción no válida. Intenta de nuevo.")


# Ejecutar
if __name__ == "__main__":
    main()


=== Gestor de Cumpleaños ===
Introduce tu correo electrónico: ian37420@gmail.com
Introduce tu contraseña: khnq jecg npbp iwwq

Opciones:
1. Registrar Cumpleaños
2. Ver Próximos Cumpleaños
3. Gestionar Mensajes
4. Enviar Correos de Hoy
5. Salir
Elige una opción: 1
Nombre: Ian
Fecha de Nacimiento (YYYY-MM-DD): 2004-11-30
Correo Electrónico: ian.castellanos1223@alumnos.udg.mx
Cumpleaños registrado con éxito.

Opciones:
1. Registrar Cumpleaños
2. Ver Próximos Cumpleaños
3. Gestionar Mensajes
4. Enviar Correos de Hoy
5. Salir
Elige una opción: 2
  Name   Birthdate  Days Until
0  Ian  2004-11-30           0

Opciones:
1. Registrar Cumpleaños
2. Ver Próximos Cumpleaños
3. Gestionar Mensajes
4. Enviar Correos de Hoy
5. Salir
Elige una opción: 3
Mensajes actuales:
Escribe un nuevo mensaje (o presiona Enter para omitir): Hooolaaaaa
Mensaje agregado.

Opciones:
1. Registrar Cumpleaños
2. Ver Próximos Cumpleaños
3. Gestionar Mensajes
4. Enviar Correos de Hoy
5. Salir
Elige una opción: 3
Mensajes a